In [1]:
# @ author: Ngagne Diop
#   date : 2021, December 27
# Import data analysis modules
import pandas as pd
import numpy as np

# I-1- Energy type for car fleet in Hauts-de-France

In [2]:
# French car fleet
P = pd.read_csv("/home/ndiop/Documents/phd/scripts/parc_vp_communes/Parc_VP_Communes_2021.csv", sep=";", encoding="utf-8")

/home/ndiop/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,2,4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# type of energy of the vehicles
P.Energie.value_counts()

Gazole                          174529
Essence                         139043
Gaz                              26205
Inconnu                          23523
Electrique et hydrogène          22862
Essence hybride rechargeable     17176
Gazole hybride rechargeable       2274
Name: Energie, dtype: int64

In [4]:
# Hauts-de-France car fleet
isHdF=P["CodeRegion"]==32
H=P[isHdF]

In [7]:
D=H[H["Energie"]=="Gazole"]
E=H[H["Energie"]=="Essence"]
G=H[H["Energie"]=="Gaz"]
I=H[H["Energie"]=="Inconnu"]
GG=H[H["Energie"]=="Electrique et hydrogène"]
II=H[H["Energie"]=="Essence hybride rechargeable"]

In [8]:
d = D["Parc01012021"].sum()/H["Parc01012021"].sum()
e = E["Parc01012021"].sum()/H["Parc01012021"].sum()
g = G["Parc01012021"].sum()/H["Parc01012021"].sum()
i = I["Parc01012021"].sum()/H["Parc01012021"].sum()
gg= GG["Parc01012021"].sum()/H["Parc01012021"].sum()
ii= II["Parc01012021"].sum()/H["Parc01012021"].sum()

In [10]:
print(d,e,g,i,gg,ii) # Diesel and Essence for 98% of energy type (60% for diesel and 38% for essence)

0.6092645773144453 0.37686269065642475 0.0053291803293997585 6.252684022416263e-05 0.005081395676005217 0.0032608201710188373


# I-2- Emission calcul

In [11]:
# Recovery of the total distance traveled from MATSim standard simulation
T1 = pd.read_csv("/home/ndiop/Documents/phd/simul/ide/matsim-code-examples/scenarios/mel/baseCase/output_trips.csv", sep=";",encoding="utf-8")
d1 = T1.groupby(['person'])['traveled_distance'].sum().reset_index()
d2=d1.rename(columns={"person":"id"})

In [13]:
# Recovery of the mode and class of income of the persons from recupTripModeCat.awk
O1 = pd.read_csv("/home/ndiop/Documents/phd/scripts/emission/outbasecaseMELok.csv", sep=";",encoding="utf-8")
O2=O1.drop_duplicates(subset=['id'], keep='last')
O2.reset_index(drop=True, inplace=True)

In [14]:
# Joining of base modes O1 and distances travelled d1
T2 = pd.merge(d2.copy(), O2.copy(), how='left',on ="id")

In [15]:
# emissions events from MATSim emission extension for base scenario with recupEmission.awk
E1 = pd.read_csv("/home/ndiop/Documents/phd/simul/ide/matsim-code-examples/emissionsBaseMEL.csv", sep=",",encoding="utf-8")

# sum of emissions per person
E2 = E1.copy().groupby(['id']).sum().reset_index()

/home/ndiop/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [16]:
# Récupération des énergie des véhicules
N = pd.read_csv("/home/ndiop/Documents/phd/simul/ide/matsim-code-examples/energyVehiclesMEL.csv", sep=",",encoding="utf-8")

In [17]:
# Energy affectation N ti vehicles E2
T3 = pd.merge(E2.copy(), N.copy(), how='left',on ="id")

# Joining T3 emissions and T2 trip data
T4 = pd.merge(T3.copy(),T2.copy(), how='left',on ="id")

T5 = T4.copy()['id'].str.split('_').str[0]
T5 = T5.to_frame()
T5 = T5.rename(columns = {'id':'person'})
T6 = pd.concat([T5.copy(),T4.copy()], axis=1, ignore_index=False)

In [18]:
# emissions per person
T6.to_csv("/home/ndiop/Documents/phd/simul/ide/matsim-code-examples/scenarios/mel/baseCase/emissionperPerson.csv", sep=";", encoding="utf8", index=False)

T7=T6[T6["mode"]=="car"] # used for emission per cat and energy
# emissions per person
T7.to_csv("/home/ndiop/Documents/phd/simul/ide/matsim-code-examples/scenarios/mel/baseCase/emissionperPersonCar.csv", sep=";", encoding="utf8", index=False)

In [19]:
T9=T7[(T7["nrj"]=="car_diesel")|(T7["nrj"]=="car_petrol")]
# emissions per person
T9.to_csv("/home/ndiop/Documents/phd/simul/ide/matsim-code-examples/scenarios/mel/baseCase/emissionperPersonNRJ.csv", sep=";", encoding="utf8", index=False)

In [20]:
# sum of emissions per person
T10 = T9.copy().groupby(['cat','nrj']).sum().reset_index()

# emissions per person and income class
T10.to_csv("/home/ndiop/Documents/phd/simul/ide/matsim-code-examples/scenarios/mel/baseCase/emissionResult.csv", sep=";", encoding="utf8", index=False)
T10[["cat","nrj","fc","co2","pmx"]]

,cat,nrj,fc,co2,pmx
0,rev1,car_diesel,7.640368e+06,2.325544e+07,1616.369496
1,rev1,car_petrol,5.036509e+06,1.533978e+07,1067.130670
2,rev2,car_diesel,3.648257e+07,1.111350e+08,7676.393131
3,rev2,car_petrol,2.393692e+07,7.291687e+07,5037.123318
4,rev3,car_diesel,2.933333e+07,8.940657e+07,6273.086715
5,rev3,car_petrol,1.947553e+07,5.933012e+07,4165.431237
